Il faut :
- prendre nos datas
- créer les emmbeddings avec la partie texte (extraire)
- faire une fonction pour trouver les similarités - faire ressortir les 3 documents les plus similaires par exemple
- fine tuner ensuite sur nos données- https://huggingface.co/blog/how-to-train-sentence-transformers
https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2
https://www.codegram.com/blog/finding-similar-documents-with-transformers/
https://huggingface.co/facebook/bart-large-mnli
https://huggingface.co/docs/transformers/tasks/question_answering

% pip install -U sentence-transformers

In [1]:
from typing import List

## Extraire le contenu du csv

In [2]:
import csv
with open("JADe_project1/Data/movie_synopsis.csv") as f:
    content = []
    csv_reader = csv.reader(f)
    for line in csv_reader:
        # process each line
        if line[0] == "":
            print(line)
        else:
            content.append(line)

FileNotFoundError: [Errno 2] No such file or directory: 'JADe_project1/Data/movie_synopsis.csv'

## User Input

In [5]:
input_ju = "C'est l'histoire d'une fille qui rencontre un garçon. Ils sont lycéens. Au début ils ne s'aiment pas du tout, et après ils deviennent meilleurs amis. "
input_jus = model.encode(input_ju)

## Télécharger le modèle et créer les vecteurs

In [30]:
from sentence_transformers import SentenceTransformer
sentences = []
for id,synopsis,title in content:
    # on récupère seulement le synopsis
    sentences.append(synopsis)

# chargement du modèle
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
embeddings = model.encode(sentences)

## Créer un objet contenant les vecteurs et les titres

In [99]:
from collections import namedtuple

# Définir un named tuple pour représenter des coordonnées géographiques
Vecteurs = namedtuple('Vecteurs', ['embedding', 'titre'])

# Créer une instance de named tuple
total = []
for em,(id,synopsis,title) in zip(embeddings,content):
    total.append(Vecteurs(embedding=em, titre=title))

#print(total[1])

## Fonction qui donne les suggestion
Par défaut, on donne 5 suggestions. 

In [97]:
from sklearn.metrics.pairwise import cosine_distances
def give_suggestion(user_input:str,k:int=5):
    # on encode l'input de l'user avec le même modèle pour avoir un vecteur
    embed_user = model.encode(user_input)
    stock_similarity = []
    stock_suggest_k = []
    # on calcule les distances cosinus
    for embed,title in total:
        result = (cosine_distances([embed],[embed_user]),title)
        # elles sont stockés dans la liste sous forme de tuple contenant la distance cosinus et le titre
        stock_similarity.append(result)
    # on trie (ordre croissant) pour avoir le synopsis le plus proche de l'input de l'usr   
    ma_liste_triee = sorted(stock_similarity, key=lambda x: x[0])
    # afficher les premiers résultats etc - on peut changer cette partie en fonction de ce qu'on veut afficher 
    #(idem pour le return)
    for i, (x, y) in enumerate(ma_liste_triee[:5]):
        print(f"Suggestion {i+1}: {y} - distance {x}")
        stock_suggest_k.append(y)
    return stock_similarity #ou stock_suggest_k ou stock_similarity[:k]

In [98]:
# Test
give_suggestion("C'est l'histoire d'adulte qui parlent et qui s'amusent.")

Suggestion 1: The one I love - distance [[0.5092616]]
Suggestion 2: Magique ! - distance [[0.5539975]]
Suggestion 3: Célibataires - distance [[0.5695218]]
Suggestion 4: Une  approche positive du comportement des perroquets - distance [[0.6182323]]
Suggestion 5: Marie Antoinette - distance [[0.69943404]]


[(array([[1.0623611]], dtype=float32), 'Four Rooms'),
 (array([[1.0410045]], dtype=float32), 'Judgment Night'),
 (array([[0.96577424]], dtype=float32), 'Life in Loops (A Megacities RMX)'),
 (array([[0.96851665]], dtype=float32), 'Star Wars'),
 (array([[1.0307138]], dtype=float32), 'Finding Nemo'),
 (array([[0.99868214]], dtype=float32), 'Forrest Gump'),
 (array([[0.97704715]], dtype=float32), 'American Beauty'),
 (array([[0.9240859]], dtype=float32), 'Citizen Kane'),
 (array([[0.94626504]], dtype=float32), 'Dancer in the Dark'),
 (array([[0.9428842]], dtype=float32), 'The Dark'),
 (array([[0.9366697]], dtype=float32), 'The Fifth Element'),
 (array([[0.9554125]], dtype=float32), 'My Life Without Me'),
 (array([[0.9733745]], dtype=float32), 'The Endless Summer'),
 (array([[0.94469815]], dtype=float32),
  'Pirates of the Caribbean: The Curse of the Black Pearl'),
 (array([[0.97206485]], dtype=float32), 'Kill Bill: Vol. 1'),
 (array([[1.066306]], dtype=float32), 'Jarhead'),
 (array([[0.921